In [2]:
#Preventing aspect: 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
df_churn = pd.read_csv("ifood_df.csv")

In [3]:
df_churn.head(3)

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,1,0,0,1529,1441,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,1,0,0,21,15,0
2,71613.0,0,0,26,426,49,127,111,21,42,...,1,0,0,0,1,0,0,734,692,0


In [4]:
np.random.seed(42)
y = df_churn['Response']  
X = df_churn.drop('Response',axis = 1)
xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size = 0.7)

In [5]:
from sklearn.linear_model import LogisticRegression
X = pd.get_dummies(X, drop_first=True)

# Split data into training and testing sets
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.7, random_state=42)

# Train logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(xTrain, yTrain)

# Print model accuracy on test set
print("Model accuracy on test set:", log_reg.score(xTest, yTest))

Model accuracy on test set: 0.8678756476683938


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
!pip install gurobipy
from gurobipy import Model, GRB, quicksum


In [7]:
pip install gurobipy==11.0.0


Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data setup
np.random.seed(42)
y = df_churn['Response'].values
X = df_churn.drop('Response', axis=1).values

# Splitting the data
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.7, random_state=42)

# Standardizing the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Parameters for robust optimization
alpha = 1.0  # Regularization parameter
delta = 0.1  # Robustness parameter (perturbation)

# Initialize Gurobi Model
model = Model("RobustLogisticRegression")

# Variables
n, p = xTrain.shape
beta = model.addVars(p, lb=-GRB.INFINITY, name="beta")  # Coefficients
b0 = model.addVar(lb=-GRB.INFINITY, name="b0")  # Intercept
z = model.addVars(n, vtype=GRB.CONTINUOUS, lb=0, name="z")  # Slack variables for robust constraints

# Objective: Minimize the sum of logistic loss and L2 regularization
log_loss = quicksum(z[i] for i in range(n))
regularization = alpha * quicksum(beta[j] * beta[j] for j in range(p))
model.setObjective(log_loss + regularization, GRB.MINIMIZE)

# Constraints
for i in range(n):
    xi = xTrain[i]
    yi = yTrain[i]
    prob_constraint = yi * (quicksum(xi[j] * beta[j] for j in range(p)) + b0)
    model.addConstr(z[i] >= 1 - prob_constraint - delta * quicksum(abs(xi[j]) * beta[j] for j in range(p)))

# Optimize
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    coefficients = np.array([beta[j].X for j in range(p)])
    intercept = b0.X
    print("Optimal Coefficients:", coefficients)
    print("Optimal Intercept:", intercept)

    # Predict on Test Set
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    yPred = sigmoid(np.dot(xTest, coefficients) + intercept)
    yPredClass = (yPred > 0.5).astype(int)
    accuracy = np.mean(yPredClass == yTest)
    print("Test Set Accuracy:", accuracy)
else:
    print("Optimization did not converge.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.2.0 22C65)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 661 rows, 700 columns and 24564 nonzeros
Model fingerprint: 0x2b61c8e8
Model has 38 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-05, 1e+01]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 107 rows and 110 columns
Presolve time: 0.01s
Presolved: 554 rows, 590 columns, 20498 nonzeros
Presolved model has 36 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 36
 AA' NZ     : 1.532e+05
 Factor NZ  : 1.537e+05 (roughly 2 MB of memory)
 Factor Ops : 5.683e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Ti

In [8]:
n_features = xTrain.shape[1]
n_samples = xTrain.shape[0]

# Create Gurobi model
model = Model("RobustLogisticRegression")
beta = model.addVars(n_features, lb=-GRB.INFINITY, name="beta")

# Define the objective using a direct approximation for log-loss
log_losses = []
for i in range(n_samples):
    y_i = yTrain.iloc[i]
    x_i = xTrain.iloc[i].values
    
    # Linear combination: beta * x_i
    lin_comb = quicksum(beta[j] * x_i[j] for j in range(n_features))
    
    # Introduce an auxiliary variable for the negative log-likelihood
    log_loss = model.addVar(lb=0, name=f"log_loss_{i}")
    
    # Add constraint: log_loss >= max(0, -y_i * lin_comb)
    model.addConstr(log_loss >= 0)
    model.addConstr(log_loss >= -y_i * lin_comb)
    
    # Approximate log(1 + exp(-y_i * (x_i * beta))) using piecewise linear constraints if necessary
    
    log_losses.append(log_loss)

# Objective: Minimize the sum of log-losses
model.setObjective(quicksum(log_losses), GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Extract optimized coefficients
optimized_beta = np.array([beta[j].X for j in range(n_features)])
print("Optimized beta coefficients:", optimized_beta)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-27
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.2.0 22C65)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1322 rows, 699 columns and 3762 nonzeros
Model fingerprint: 0x002e17a3
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1322 rows and 699 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   3.800000e-05      0s
Extra simplex iterations after uncrush: 31
      31    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Optimized beta coefficients: [0. 0. 0. 0.

In [9]:
import gurobipy as gp